# Spotify Audio Features

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from requests.exceptions import ReadTimeout

In [2]:
from tqdm.notebook import tqdm
#from tqdm import tqdm

In [3]:
import pandasql as ps

In [4]:
import pandas as pd
import numpy as np
import json
import ast
import datetime
import time

import warnings
warnings.filterwarnings("ignore")


In [67]:
#Authentication - without user

# Input client ID and client secret here
# Client ID and Client Secret
# Coming from https://developer.spotify.com/dashboard/

# This particular is called from spotify_app:
# https://developer.spotify.com/dashboard/applications/YOUR_APPLICATION
cid = 'INSERT_CLIENT_ID'
secret = 'INSERT_CLIENT_SECTRET'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, 
                                                      client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout=10, retries=10)

In [6]:
# The MyData/StreamingHistory{k}.json is the streaming history provided by the spotify client
# The data is retrieved by going to your spotify account on Spotify.com:
## https://www.spotify.com/dk-da/account/privacy/
file_in = 'MyData/StreamingHistory3.json'
# Determine the name of the enriched csv-file we will end up using this notebook
file_out = 'StreamingHistory3_enriched.csv'

In [7]:
# Open the file_in - in this case 'MyData/StreamingHistory3.json'
with open(file_in) as json_file:
    data = json.load(json_file)
df = pd.json_normalize(data)
df.head()

,endTime,artistName,trackName,msPlayed
0,2021-12-30 19:05,Sickick,I Can Feel It,129258
1,2021-12-30 19:06,Drake,Way 2 Sexy (with Future & Young Thug),38165
2,2021-12-30 19:06,Gunna,too easy,4224
3,2021-12-30 19:06,Future,Life Is Good (feat. Drake),6506
4,2021-12-30 19:07,Drake,Way 2 Sexy (with Future & Young Thug),15935


In [9]:
# Check how many records
df.shape

(9628, 4)

In [10]:
# Fast checking whether or not our for-loop will work
for row in range(len(df.head(5))):
    artist = df.iloc[row]['artistName']
    track = df.iloc[row]['trackName']
    print(track )

I Can Feel It
Way 2 Sexy (with Future & Young Thug)
too easy
Life Is Good (feat. Drake)
Way 2 Sexy (with Future & Young Thug)


In [11]:
# We had some problems with the amount of calls we could send to the Spotify API
# Thus, we will divide the calls by two and call in batches (ish)
a_ = int(len(df)/2)
range(a_)

range(0, 4814)

In [12]:
# Check the min range 
# (We're doing a concatenation with 'ignore_index=True' later, so we want to make sure the correct rows match)
np.min(range(a_))

0

In [13]:
# Check the max range
np.max(range(a_))

4813

In [14]:
# Define resulting dataframe
df_track = pd.DataFrame(
    columns=[
        'endTime',
        'track_id',
        'track_name',
        'track_artist',
        'genres',
        'msPlayed'
    ]
)

# Iterator variable to introduce a sleep timer per calling 100 songs (in the end of this cell)
i = 0
# We had some problems with the amount of calls we could send to the Spotify API
# Thus, we will divide the calls by two and call in batches (ish)
a_ = int(len(df)/2)
for row in tqdm(range(a_)):
    endTime = df.iloc[row]['endTime']
    artist = df.iloc[row]['artistName']
    track = df.iloc[row]['trackName']
    msPlayed = df.iloc[row]['msPlayed']

    # Search the Spotify API for the artist and track to be able to get the following rows out
    # 'endTime', 'track_id', 'track_name', 'track_artist', 'genres', 'msPlayed'
    # So, basically, we want to add genres and the track_id of the song (to be used later in this notebook)

    # All the try/except statements are for catching errors
    try:
        track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=10)
    except ReadTimeout:
        print('Spotify timed out... trying again...')
        break
        track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=10)
        
    search_result = pd.json_normalize(track_q['tracks']['items'])
    # Get the track id (top result) the search query 
    search_result = search_result.head(1)
    try:
        track_id = search_result.id.values[0]
    except:
        track_id = None
    try:
        track_name = search_result.name.values[0]
    except:
        track_name = None
    try:
        track_artist  = search_result.artists.values[0][0]['name']
    except:
        track_artist = None
        
    if track_artist:
        try:
            artist_q = sp.artist(search_result.artists[0][0]['external_urls']["spotify"])
            genres  = artist_q['genres']
        except:
            genres = None
    else:
        genres = None
    
    # Add the newly found data to a dataframe
    df_track = df_track.append(
        {
            'endTime':endTime,
            'track_id':track_id,
            'track_name':track_name,
            'track_artist':track_artist,
            'genres':genres,
            'msPlayed':msPlayed
        }, ignore_index=True
    )
    
    i += 1

    if i == 100:
        time.sleep(10)
        i = 0
    
#df_track
#search_result

HTTP Error for GET to https://api.spotify.com/v1/artists/2TzHIUhVpeeDxyJPpQfnV3 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/1p2z4hiBCkxfWZ6OvyGBSA with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/0lAWpj5szCSwM4rUMHYmrr with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/5c8JlthNH2qFzRcKIKswi6 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/6a3WaWgxrVmNIvaiTCI0Kk with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/2MvVPVskst6WSd1JWA5311 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/27eEve8Mu7Zv8DGDIz4peE with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/14bt7qAPOSPw76Fc1WFGM6 with Params: {} returned 404 due to None
HTTP Error for GET to https://ap

HTTP Error for GET to https://api.spotify.com/v1/artists/6DPYiyq5kWVQS4RGwxzPC7 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/0CbeG1224FS58EUx4tPevZ with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/11BUDylkl50Y6dsbZMZiCG with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/36Na2YYP8im6qQJBZu2fsa with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/3IamLW3k3rAlmCehXEBx1G with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/3pJK3CUpbnyX7ogDfAwa9J with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/754BJcKdHZH7tj9xFq3hHO with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/7guDJrEfX3qb6FEbdPA5qi with Params: {} returned 404 due to None
HTTP Error for GET to https://ap

In [15]:
df_track.head(10)

,endTime,track_id,track_name,track_artist,genres,msPlayed
0,2021-12-30 19:05,None,None,None,None,129258
1,2021-12-30 19:06,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",38165
2,2021-12-30 19:06,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"[atl hip hop, melodic rap, rap, trap]",4224
3,2021-12-30 19:06,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"[atl hip hop, hip hop, pop rap, rap, southern ...",6506
4,2021-12-30 19:07,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",15935
5,2021-12-30 19:07,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"[atl hip hop, hip hop, pop rap, rap, southern ...",59520
6,2021-12-31 09:54,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",746
7,2021-12-31 09:54,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",16128
8,2021-12-31 10:16,5qaaQp6KvkEdB2q7SKQAcJ,ErruDumEllaHvad,ICEKIID,"[danish hip hop, danish pop]",37109
9,2021-12-31 10:16,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,"[lgbtq+ hip hop, pop]",211999


In [16]:
df.iloc[np.min(range(a_))]

endTime       2021-12-30 19:05
artistName             Sickick
trackName        I Can Feel It
msPlayed                129258
Name: 0, dtype: object

In [17]:
df.iloc[np.max(range(a_))]

endTime       2022-02-11 10:55
artistName         Miles Davis
trackName                Rouge
msPlayed                195685
Name: 4813, dtype: object

In [ ]:
df_track

In [18]:
df.iloc[np.min(range(a_, len(df)))]

endTime         2022-02-11 11:02
artistName      Sanne Salomonsen
trackName     Kærligheden Kalder
msPlayed                    5108
Name: 4814, dtype: object

In [19]:
df.iloc[np.max(range(a_, len(df)))]

endTime       2022-04-01 19:20
artistName            Larry 44
trackName               T-Pain
msPlayed                120225
Name: 9627, dtype: object

In [20]:
#df_track.shape

In [21]:
#df_track.head(5)

In [22]:
#df_track.tail()

In [23]:
#df_track

In [24]:
#df.shape

In [25]:
#range(a_ + 1, len(df))

In [26]:
#np.min(range(a_ , len(df)))

In [27]:
#np.max(range(a_, len(df)))

In [31]:
df_track_2 = pd.DataFrame(
    columns=[
        'endTime',
        'track_id',
        'track_name',
        'track_artist',
        'genres',
        'msPlayed'
    ]
)

# Iterator variable to introduce a sleep timer per calling 100 songs (in the end of this cell)
i = 0
# We had some problems with the amount of calls we could send to the Spotify API
# Thus, we will divide the calls by two and call in batches (ish)
for row in tqdm(range(a_, len(df))):
    endTime = df.iloc[row]['endTime']
    artist = df.iloc[row]['artistName']
    track = df.iloc[row]['trackName']
    msPlayed = df.iloc[row]['msPlayed']

    # Search the Spotify API for the artist and track to be able to get the following rows out
    # 'endTime', 'track_id', 'track_name', 'track_artist', 'genres', 'msPlayed'
    # So, basically, we want to add genres and the track_id of the song (to be used later in this notebook)

    # All the try/except statements are for catching errors

    try:
        track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=10)
    except ReadTimeout:
        print('Spotify timed out... trying again...')
        track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=10)
        
    #track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track')
    search_result = pd.json_normalize(track_q['tracks']['items'])
    # Get the track id (top result) the search query 
    search_result = search_result.head(1)
    try:
        track_id = search_result.id.values[0]
    except:
        track_id = None
    try:
        track_name = search_result.name.values[0]
    except:
        track_name = None
    try:
        track_artist  = search_result.artists.values[0][0]['name']
    except:
        track_artist = None
        
    if track_artist:
        try:
            artist_q = sp.artist(search_result.artists[0][0]['external_urls']["spotify"])
            genres  = artist_q['genres']
        except:
            genres = None
    
    # Add the newly found data to a dataframe
    df_track_2 = df_track_2.append(
        {
            'endTime':endTime,
            'track_id':track_id,
            'track_name':track_name,
            'track_artist':track_artist,
            'genres':genres,
            'msPlayed':msPlayed
        }, ignore_index=True
    )
    
    i += 1

    if i == 100:
        time.sleep(10)
        i = 0
        

HTTP Error for GET to https://api.spotify.com/v1/artists/7Ap8xCd7d0xon5XnkLvn4i with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/7nqVEhD14JLieCcN2n3CiB with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/5SwsSD9zyWaWeVyK9qIW9l with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/716NhGYqD1jl2wI1Qkgq36 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/7wrulS1dfanckBnoxxEuS6 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/07QEuhtrNmmZ0zEcqE9SF6 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/74HomvqoUecDcUPRespyqg with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/6A8VCzSDHW4C4Q2J0sIJJN with Params: {} returned 404 due to None
HTTP Error for GET to https://ap

HTTP Error for GET to https://api.spotify.com/v1/artists/3TOqt5oJwL9BE2NG9MEwDa with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/3PhoLpVuITZKcymswpck5b with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/6Ha4aES39QiVjR0L2lwuwq with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/1z7b1Pr1rSlvWRzsW3HOrS with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/6LuN9FCkKOj5PcnpouEgny with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/4RbUYWWjEBb4umwqakOEd3 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/6GTwMrB4u3hwcUyc9sU1UL with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/2tIP7SsRs7vjIcLrU85W8J with Params: {} returned 404 due to None
HTTP Error for GET to https://ap

KeyboardInterrupt: 

In [60]:
df_track_2

,endTime,track_id,track_name,track_artist,genres,msPlayed
0,2022-02-11 11:02,4fJyuZM1uvNQEiLMH1PQ8n,Kærligheden Kalder,Sanne Salomonsen,"[classic danish pop, danish pop, dansktop]",5108
1,2022-02-11 11:02,5ujh1I7NZH5agbwf7Hp8Hc,Swimming Pools (Drank) - Extended Version,Kendrick Lamar,"[conscious hip hop, hip hop, rap, west coast rap]",1381
2,2022-02-11 11:02,7l5In0rLyHmqJhKlzZPtYo,"Straight, No Chaser (feat. John Coltrane, Cann...",Miles Davis,"[cool jazz, hard bop, jazz, jazz fusion, jazz ...",422961
3,2022-02-11 11:02,5Pt83OUaK3K9C2if4C5JOk,Gotta Have It,JAY-Z,"[east coast hip hop, hip hop, rap]",1764
4,2022-02-11 11:05,4JC18crxRPZOuTqBfXKFIR,"MHD: Bzrp Music Sessions, Vol. 44",Bizarrap,"[argentine hip hop, pop venezolano, trap argen...",173380
...,...,...,...,...,...,...
4575,2022-03-30 20:48,3I09LQbHS3NSU46Ly3tPpR,Feeling Good,Michael Bublé,"[adult standards, canadian pop, jazz pop, lounge]",170048
4576,2022-03-31 06:39,2znzyzlAUc8Z1Ikq9aFOcx,Tuesday (feat. Drake),ILOVEMAKONNEN,"[atl hip hop, lgbtq+ hip hop, trap]",789
4577,2022-03-31 06:39,3I09LQbHS3NSU46Ly3tPpR,Feeling Good,Michael Bublé,"[adult standards, canadian pop, jazz pop, lounge]",8426
4578,2022-03-31 06:39,1ASfDW1PgHJ9g97T7B1a4J,Siger Ingenting,Hans Philip,[danish pop],1408


In [61]:
df_track.tail(1)

,endTime,track_id,track_name,track_artist,genres,msPlayed
4813,2022-02-11 10:55,3AvqJzm3zQuRIpNXhdMaAN,Rouge,Miles Davis,"[cool jazz, hard bop, jazz, jazz fusion, jazz ...",195685


In [62]:
df_track_2.tail(1)

,endTime,track_id,track_name,track_artist,genres,msPlayed
4579,2022-03-31 06:39,3DZShPBwoWAfFgPKBUcFFJ,Softly As In A Morning Sunrise - Remastered,Sonny Clark Trio,"[hard bop, jazz trio]",1194


In [63]:
4813+4579

9392

In [64]:
df.iloc[4813+4579]

endTime       2022-03-31 06:39
artistName         Hans Philip
trackName      Siger Ingenting
msPlayed                  1408
Name: 9392, dtype: object

In [65]:
df.iloc[4813+4579 + 1]

endTime                     2022-03-31 06:39
artistName                       Sonny Clark
trackName     Softly As In A Morning Sunrise
msPlayed                                1194
Name: 9393, dtype: object

In [66]:
df.iloc[4813+4579+2]

endTime         2022-03-31 06:42
artistName        Madison Avenue
trackName     Don't Call Me Baby
msPlayed                    1151
Name: 9394, dtype: object

In [68]:
df_track_3 = pd.DataFrame(
    columns=[
        'endTime',
        'track_id',
        'track_name',
        'track_artist',
        'genres',
        'msPlayed'
    ]
)

# Iterator variable to introduce a sleep timer per calling 100 songs (in the end of this cell)
i = 0
# We had some problems with the amount of calls we could send to the Spotify API
# Thus, we will divide the calls by two and call in batches (ish)
_range = range(4813+4579 + 2, len(df))
for row in tqdm(_range):
    endTime = df.iloc[row]['endTime']
    artist = df.iloc[row]['artistName']
    track = df.iloc[row]['trackName']
    msPlayed = df.iloc[row]['msPlayed']
    
    # Search the Spotify API for the artist and track to be able to get the following rows out
    # 'endTime', 'track_id', 'track_name', 'track_artist', 'genres', 'msPlayed'
    # So, basically, we want to add genres and the track_id of the song (to be used later in this notebook)

    # All the try/except statements are for catching errors

    try:
        track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=10)
    except ReadTimeout:
        print('Spotify timed out... trying again...')
        track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=10)
        
    #track_q = sp.search(q='artist:' + artist + ' track:' + track, type='track')
    search_result = pd.json_normalize(track_q['tracks']['items'])
    # Get the track id (top result) the search query 
    search_result = search_result.head(1)
    try:
        track_id = search_result.id.values[0]
    except:
        track_id = None
    try:
        track_name = search_result.name.values[0]
    except:
        track_name = None
    try:
        track_artist  = search_result.artists.values[0][0]['name']
    except:
        track_artist = None
        
    if track_artist:
        try:
            artist_q = sp.artist(search_result.artists[0][0]['external_urls']["spotify"])
            genres  = artist_q['genres']
        except:
            genres = None
    
    # Append to dataframe
    df_track_3 = df_track_3.append(
        {
            'endTime':endTime,
            'track_id':track_id,
            'track_name':track_name,
            'track_artist':track_artist,
            'genres':genres,
            'msPlayed':msPlayed
        }, ignore_index=True
    )
    
    i += 1

    if i == 100:
        time.sleep(10)
        i = 0
        
#df_track
#search_result

HTTP Error for GET to https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/3hv9jJF3adDNsBSIQDqcjp with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/5j4HeCoUlzhfWtjAfM1acR with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/2feDdbD5araYcm6JhFHHw7 with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/4ZwOlDVAGXQo66ykbLQ8dP with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/3RYYha3CC7js2PHbcBHewt with Params: {} returned 404 due to None
HTTP Error for GET to https://api.spotify.com/v1/artists/3VDPCWdHfS2OIF4ePOIeOV with Params: {} returned 404 due to None


In [69]:
# Concatenate all the results
df_track_res = pd.concat([df_track, df_track_2, df_track_3], axis=0)
df_track_res = df_track_res.reset_index(drop=True)
df_track_res

,endTime,track_id,track_name,track_artist,genres,msPlayed
0,2021-12-30 19:05,None,None,None,None,129258
1,2021-12-30 19:06,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",38165
2,2021-12-30 19:06,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"[atl hip hop, melodic rap, rap, trap]",4224
3,2021-12-30 19:06,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"[atl hip hop, hip hop, pop rap, rap, southern ...",6506
4,2021-12-30 19:07,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",15935
...,...,...,...,...,...,...
9623,2022-04-01 19:07,49QK41xxLFFPVLm3caInNB,Santé,Stromae,"[belgian pop, g-house]",190999
9624,2022-04-01 19:10,58HvfVOeJY7lUuCqF0m3ly,MIDDLE OF THE NIGHT,Elley Duhé,"[alt z, pop]",184447
9625,2022-04-01 19:13,3JiVOU9tXbRi7aE8hBaz5v,Kopenhagen (feat. ICEKIID & Noah Carter),Gilli,"[danish hip hop, danish pop]",147426
9626,2022-04-01 19:18,0lVcRfuCL9U6DCmyAJAqmV,Tag mig tilbage,Nik & Jay,"[danish hip hop, danish pop]",292665


In [71]:
# Save for future use
df_track_res.to_csv('df_track_res.csv', index=False)

In [72]:
def process_audio_features(audio_data):
    """
    This is a function for obtaining a song's audio features based on 
    track_id (which was found in the latter for-loops)
    """
    track_features = pd.DataFrame(
        columns=['danceability', 
                 'energy', 
                 'key', 
                 'loudness', 
                 'mode', 
                 'speechiness', 
                 'acousticness', 
                 'instrumentalness', 
                 'liveness', 
                 'valence', 
                 'tempo', 
                 'type', 
                 'id', 
                 'uri', 
                 'track_href', 
                 'analysis_url', 
                 'duration_ms', 
                 'time_signature'])
    
    # Casual checking commented out below
    #print('danceability :', audio_data[0]['danceability'])
    #print('energy :', audio_data[0]['energy'])
    #print('key :', audio_data[0]['key'])
    #print('loudness :', audio_data[0]['loudness'])
    #print('mode :', audio_data[0]['mode'])
    #print('speechiness :', audio_data[0]['speechiness'])
    #print('acousticness :', audio_data[0]['acousticness'])
    #print('instrumentalness :', audio_data[0]['instrumentalness'])
    #print('liveness :', audio_data[0]['liveness'])
    #print('valence :', audio_data[0]['valence'])
    #print('tempo :', audio_data[0]['tempo'])
    #print('type :', audio_data[0]['type'])
    #print('id :', audio_data[0]['id'])
    #print('uri :', audio_data[0]['uri'])
    #print('track_href :', audio_data[0]['track_href'])
    #print('analysis_url :', audio_data[0]['analysis_url'])
    #print('duration_ms :', audio_data[0]['duration_ms'])
    #print('time_signature :', audio_data[0]['time_signature'])
    
    # If no audio features was found for the track_id, return -1 on the parameters
    if audio_data == None:
        danceability = -1
        energy = -1
        key = -1
        loudness = -1
        mode = -1
        speechiness = -1
        acousticness = -1
        instrumentalness = -1
        liveness = -1
        valence = -1
        tempo = -1
        type = -1
        id = -1
        uri = -1
        track_href = -1
        analysis_url = -1
        duration_ms = -1
        time_signature = -1
    else:
        danceability = audio_data[0]['danceability']
        energy = audio_data[0]['energy']
        key = audio_data[0]['key']
        loudness = audio_data[0]['loudness']
        mode = audio_data[0]['mode']
        speechiness = audio_data[0]['speechiness']
        acousticness = audio_data[0]['acousticness']
        instrumentalness = audio_data[0]['instrumentalness']
        liveness = audio_data[0]['liveness']
        valence = audio_data[0]['valence']
        tempo = audio_data[0]['tempo']
        type = audio_data[0]['type']
        id = audio_data[0]['id']
        uri = audio_data[0]['uri']
        track_href = audio_data[0]['track_href']
        analysis_url = audio_data[0]['analysis_url']
        duration_ms = audio_data[0]['duration_ms']
        time_signature = audio_data[0]['time_signature']
        
    # Append to dataframe
    track_features = {
        'danceability': danceability,
        'energy': energy,
        'key': key,
        'loudness': loudness,
        'mode': mode,
        'speechiness': speechiness,
        'acousticness': acousticness,
        'instrumentalness': instrumentalness,
        'liveness': liveness,
        'valence': valence,
        'tempo': tempo,
        'type': type,
        'id': id,
        'uri': uri,
        'track_href': track_href,
        'analysis_url': analysis_url,
        'duration_ms': duration_ms,
        'time_signature': time_signature,
    }
    return track_features

## Go from here

In [73]:
df_track_res = pd.read_csv('df_track_res.csv')
df_track_res = df_track_res.dropna()

In [95]:
tqdm.pandas()

In [93]:
df_track_res

,endTime,track_id,track_name,track_artist,genres,msPlayed
1,2021-12-30 19:06,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165
2,2021-12-30 19:06,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224
3,2021-12-30 19:06,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",6506
4,2021-12-30 19:07,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",15935
5,2021-12-30 19:07,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",59520
...,...,...,...,...,...,...
9623,2022-04-01 19:07,49QK41xxLFFPVLm3caInNB,Santé,Stromae,"['belgian pop', 'g-house']",190999
9624,2022-04-01 19:10,58HvfVOeJY7lUuCqF0m3ly,MIDDLE OF THE NIGHT,Elley Duhé,"['alt z', 'pop']",184447
9625,2022-04-01 19:13,3JiVOU9tXbRi7aE8hBaz5v,Kopenhagen (feat. ICEKIID & Noah Carter),Gilli,"['danish hip hop', 'danish pop']",147426
9626,2022-04-01 19:18,0lVcRfuCL9U6DCmyAJAqmV,Tag mig tilbage,Nik & Jay,"['danish hip hop', 'danish pop']",292665


In [103]:
# For each track_id, find audio features 
# (returns a dictionary, that process_audio_features() will handle in the next cells)
audio_feats = []
for track_id in tqdm(df_track_res.track_id.values):
    try:
        audio_feats.append(sp.audio_features(track_id))
    except:
        audio_feats.append(None)

100%|██████████| 8962/8962 [14:48<00:00, 10.08it/s]  


In [105]:
df_track_res['audio_data'] = audio_feats
df_track_res

,endTime,track_id,track_name,track_artist,genres,msPlayed,audio_data
1,2021-12-30 19:06,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165,"[{'danceability': 0.803, 'energy': 0.597, 'key..."
2,2021-12-30 19:06,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224,"[{'danceability': 0.798, 'energy': 0.574, 'key..."
3,2021-12-30 19:06,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",6506,"[{'danceability': 0.676, 'energy': 0.609, 'key..."
4,2021-12-30 19:07,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",15935,"[{'danceability': 0.803, 'energy': 0.597, 'key..."
5,2021-12-30 19:07,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",59520,"[{'danceability': 0.752, 'energy': 0.658, 'key..."
...,...,...,...,...,...,...,...
9623,2022-04-01 19:07,49QK41xxLFFPVLm3caInNB,Santé,Stromae,"['belgian pop', 'g-house']",190999,"[{'danceability': 0.795, 'energy': 0.67, 'key'..."
9624,2022-04-01 19:10,58HvfVOeJY7lUuCqF0m3ly,MIDDLE OF THE NIGHT,Elley Duhé,"['alt z', 'pop']",184447,"[{'danceability': 0.41, 'energy': 0.611, 'key'..."
9625,2022-04-01 19:13,3JiVOU9tXbRi7aE8hBaz5v,Kopenhagen (feat. ICEKIID & Noah Carter),Gilli,"['danish hip hop', 'danish pop']",147426,"[{'danceability': 0.747, 'energy': 0.672, 'key..."
9626,2022-04-01 19:18,0lVcRfuCL9U6DCmyAJAqmV,Tag mig tilbage,Nik & Jay,"['danish hip hop', 'danish pop']",292665,"[{'danceability': 0.776, 'energy': 0.893, 'key..."


In [111]:
# Process the audio features
df_temp = df_track_res.audio_data.apply(
    lambda x: process_audio_features(x)
)

In [112]:
df_temp

1       {'danceability': 0.803, 'energy': 0.597, 'key'...
2       {'danceability': 0.798, 'energy': 0.574, 'key'...
3       {'danceability': 0.676, 'energy': 0.609, 'key'...
4       {'danceability': 0.803, 'energy': 0.597, 'key'...
5       {'danceability': 0.752, 'energy': 0.658, 'key'...
                              ...                        
9623    {'danceability': 0.795, 'energy': 0.67, 'key':...
9624    {'danceability': 0.41, 'energy': 0.611, 'key':...
9625    {'danceability': 0.747, 'energy': 0.672, 'key'...
9626    {'danceability': 0.776, 'energy': 0.893, 'key'...
9627    {'danceability': 0.876, 'energy': 0.518, 'key'...
Name: audio_data, Length: 8962, dtype: object

In [113]:
# list of dictionaries to dataframe
df_temp = pd.DataFrame(list(df_temp))

In [114]:
df_temp.reset_index(drop=True, inplace=True)

In [115]:
df_track_res.reset_index(drop=True, inplace=True)

In [116]:
# Add to the dataframe previously obtained
df_res = pd.concat([df_track_res, df_temp], axis=1, ignore_index=False)

In [117]:
df_res

,endTime,track_id,track_name,track_artist,genres,msPlayed,audio_data,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2021-12-30 19:06,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,0.3230,0.3310,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4
1,2021-12-30 19:06,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224,"[{'danceability': 0.798, 'energy': 0.574, 'key...",0.798,0.574,1,...,0.1220,0.3090,155.964,audio_features,4eL3XeuGaEoVT8ttDh3hwY,spotify:track:4eL3XeuGaEoVT8ttDh3hwY,https://api.spotify.com/v1/tracks/4eL3XeuGaEoV...,https://api.spotify.com/v1/audio-analysis/4eL3...,138587,4
2,2021-12-30 19:06,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",6506,"[{'danceability': 0.676, 'energy': 0.609, 'key...",0.676,0.609,2,...,0.1520,0.5080,142.037,audio_features,5yY9lUy8nbvjM1Uyo1Uqoc,spotify:track:5yY9lUy8nbvjM1Uyo1Uqoc,https://api.spotify.com/v1/tracks/5yY9lUy8nbvj...,https://api.spotify.com/v1/audio-analysis/5yY9...,237735,4
3,2021-12-30 19:07,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",15935,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,0.3230,0.3310,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4
4,2021-12-30 19:07,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",59520,"[{'danceability': 0.752, 'energy': 0.658, 'key...",0.752,0.658,0,...,0.0927,0.3870,151.951,audio_features,0k7wmahjkn389wAZdz19Cv,spotify:track:0k7wmahjkn389wAZdz19Cv,https://api.spotify.com/v1/tracks/0k7wmahjkn38...,https://api.spotify.com/v1/audio-analysis/0k7w...,213520,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8957,2022-04-01 19:07,49QK41xxLFFPVLm3caInNB,Santé,Stromae,"['belgian pop', 'g-house']",190999,"[{'danceability': 0.795, 'energy': 0.67, 'key'...",0.795,0.670,0,...,0.0414,0.8440,137.920,audio_features,49QK41xxLFFPVLm3caInNB,spotify:track:49QK41xxLFFPVLm3caInNB,https://api.spotify.com/v1/tracks/49QK41xxLFFP...,https://api.spotify.com/v1/audio-analysis/49QK...,191000,3
8958,2022-04-01 19:10,58HvfVOeJY7lUuCqF0m3ly,MIDDLE OF THE NIGHT,Elley Duhé,"['alt z', 'pop']",184447,"[{'danceability': 0.41, 'energy': 0.611, 'key'...",0.410,0.611,4,...,0.1160,0.0899,185.727,audio_features,58HvfVOeJY7lUuCqF0m3ly,spotify:track:58HvfVOeJY7lUuCqF0m3ly,https://api.spotify.com/v1/tracks/58HvfVOeJY7l...,https://api.spotify.com/v1/audio-analysis/58Hv...,184448,3
8959,2022-04-01 19:13,3JiVOU9tXbRi7aE8hBaz5v,Kopenhagen (feat. ICEKIID & Noah Carter),Gilli,"['danish hip hop', 'danish pop']",147426,"[{'danceability': 0.747, 'energy': 0.672, 'key...",0.747,0.672,11,...,0.1250,0.7660,99.719,audio_features,3JiVOU9tXbRi7aE8hBaz5v,spotify:track:3JiVOU9tXbRi7aE8hBaz5v,https://api.spotify.com/v1/tracks/3JiVOU9tXbRi...,https://api.spotify.com/v1/audio-analysis/3JiV...,147427,4
8960,2022-04-01 19:18,0lVcRfuCL9U6DCmyAJAqmV,Tag mig tilbage,Nik & Jay,"['danish hip hop', 'danish pop']",292665,"[{'danceability': 0.776, 'energy': 0.893, 'key...",0.776,0.893,6,...,0.0582,0.9680,89.021,audio_features,0lVcRfuCL9U6DCmyAJAqmV,spotify:track:0lVcRfuCL9U6DCmyAJAqmV,https://api.spotify.com/v1/tracks/0lVcRfuCL9U6...,https://api.spotify.com/v1/audio-analysis/0lVc...,292667,4


In [118]:
#sp.audio_analysis(track_id)

In [119]:
df_StreamingHistory0 = df_res #pd.concat([df, df_res], axis=1, ignore_index=False)

In [120]:
df_StreamingHistory0

,endTime,track_id,track_name,track_artist,genres,msPlayed,audio_data,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2021-12-30 19:06,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,0.3230,0.3310,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4
1,2021-12-30 19:06,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224,"[{'danceability': 0.798, 'energy': 0.574, 'key...",0.798,0.574,1,...,0.1220,0.3090,155.964,audio_features,4eL3XeuGaEoVT8ttDh3hwY,spotify:track:4eL3XeuGaEoVT8ttDh3hwY,https://api.spotify.com/v1/tracks/4eL3XeuGaEoV...,https://api.spotify.com/v1/audio-analysis/4eL3...,138587,4
2,2021-12-30 19:06,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",6506,"[{'danceability': 0.676, 'energy': 0.609, 'key...",0.676,0.609,2,...,0.1520,0.5080,142.037,audio_features,5yY9lUy8nbvjM1Uyo1Uqoc,spotify:track:5yY9lUy8nbvjM1Uyo1Uqoc,https://api.spotify.com/v1/tracks/5yY9lUy8nbvj...,https://api.spotify.com/v1/audio-analysis/5yY9...,237735,4
3,2021-12-30 19:07,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",15935,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,0.3230,0.3310,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4
4,2021-12-30 19:07,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",59520,"[{'danceability': 0.752, 'energy': 0.658, 'key...",0.752,0.658,0,...,0.0927,0.3870,151.951,audio_features,0k7wmahjkn389wAZdz19Cv,spotify:track:0k7wmahjkn389wAZdz19Cv,https://api.spotify.com/v1/tracks/0k7wmahjkn38...,https://api.spotify.com/v1/audio-analysis/0k7w...,213520,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8957,2022-04-01 19:07,49QK41xxLFFPVLm3caInNB,Santé,Stromae,"['belgian pop', 'g-house']",190999,"[{'danceability': 0.795, 'energy': 0.67, 'key'...",0.795,0.670,0,...,0.0414,0.8440,137.920,audio_features,49QK41xxLFFPVLm3caInNB,spotify:track:49QK41xxLFFPVLm3caInNB,https://api.spotify.com/v1/tracks/49QK41xxLFFP...,https://api.spotify.com/v1/audio-analysis/49QK...,191000,3
8958,2022-04-01 19:10,58HvfVOeJY7lUuCqF0m3ly,MIDDLE OF THE NIGHT,Elley Duhé,"['alt z', 'pop']",184447,"[{'danceability': 0.41, 'energy': 0.611, 'key'...",0.410,0.611,4,...,0.1160,0.0899,185.727,audio_features,58HvfVOeJY7lUuCqF0m3ly,spotify:track:58HvfVOeJY7lUuCqF0m3ly,https://api.spotify.com/v1/tracks/58HvfVOeJY7l...,https://api.spotify.com/v1/audio-analysis/58Hv...,184448,3
8959,2022-04-01 19:13,3JiVOU9tXbRi7aE8hBaz5v,Kopenhagen (feat. ICEKIID & Noah Carter),Gilli,"['danish hip hop', 'danish pop']",147426,"[{'danceability': 0.747, 'energy': 0.672, 'key...",0.747,0.672,11,...,0.1250,0.7660,99.719,audio_features,3JiVOU9tXbRi7aE8hBaz5v,spotify:track:3JiVOU9tXbRi7aE8hBaz5v,https://api.spotify.com/v1/tracks/3JiVOU9tXbRi...,https://api.spotify.com/v1/audio-analysis/3JiV...,147427,4
8960,2022-04-01 19:18,0lVcRfuCL9U6DCmyAJAqmV,Tag mig tilbage,Nik & Jay,"['danish hip hop', 'danish pop']",292665,"[{'danceability': 0.776, 'energy': 0.893, 'key...",0.776,0.893,6,...,0.0582,0.9680,89.021,audio_features,0lVcRfuCL9U6DCmyAJAqmV,spotify:track:0lVcRfuCL9U6DCmyAJAqmV,https://api.spotify.com/v1/tracks/0lVcRfuCL9U6...,https://api.spotify.com/v1/audio-analysis/0lVc...,292667,4


In [121]:
df_StreamingHistory0.to_csv(file_out, index=False)

In [122]:
df_StreamingHistory0.isnull().sum() / len(df_StreamingHistory0)

endTime             0.0
track_id            0.0
track_name          0.0
track_artist        0.0
genres              0.0
msPlayed            0.0
audio_data          0.0
danceability        0.0
energy              0.0
key                 0.0
loudness            0.0
mode                0.0
speechiness         0.0
acousticness        0.0
instrumentalness    0.0
liveness            0.0
valence             0.0
tempo               0.0
type                0.0
id                  0.0
uri                 0.0
track_href          0.0
analysis_url        0.0
duration_ms         0.0
time_signature      0.0
dtype: float64

In [123]:
df_StreamingHistory0.endTime = pd.to_datetime(df_StreamingHistory0.endTime)

In [124]:
#df_StreamingHistory0['endTime_timeStamp'] = df_StreamingHistory0.endTime.apply(lambda x: x.timestamp()).copy()

In [125]:
df_StreamingHistory0.head(5)

,endTime,track_id,track_name,track_artist,genres,msPlayed,audio_data,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,2021-12-30 19:06:00,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,0.3230,0.331,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4
1,2021-12-30 19:06:00,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224,"[{'danceability': 0.798, 'energy': 0.574, 'key...",0.798,0.574,1,...,0.1220,0.309,155.964,audio_features,4eL3XeuGaEoVT8ttDh3hwY,spotify:track:4eL3XeuGaEoVT8ttDh3hwY,https://api.spotify.com/v1/tracks/4eL3XeuGaEoV...,https://api.spotify.com/v1/audio-analysis/4eL3...,138587,4
2,2021-12-30 19:06:00,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",6506,"[{'danceability': 0.676, 'energy': 0.609, 'key...",0.676,0.609,2,...,0.1520,0.508,142.037,audio_features,5yY9lUy8nbvjM1Uyo1Uqoc,spotify:track:5yY9lUy8nbvjM1Uyo1Uqoc,https://api.spotify.com/v1/tracks/5yY9lUy8nbvj...,https://api.spotify.com/v1/audio-analysis/5yY9...,237735,4
3,2021-12-30 19:07:00,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",15935,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,0.3230,0.331,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4
4,2021-12-30 19:07:00,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",59520,"[{'danceability': 0.752, 'energy': 0.658, 'key...",0.752,0.658,0,...,0.0927,0.387,151.951,audio_features,0k7wmahjkn389wAZdz19Cv,spotify:track:0k7wmahjkn389wAZdz19Cv,https://api.spotify.com/v1/tracks/0k7wmahjkn38...,https://api.spotify.com/v1/audio-analysis/0k7w...,213520,4


# Add start time (column) 

In [126]:
df = pd.read_csv(file_out)

In [127]:
#df_StreamingHistory0

In [128]:
#df.columns

In [129]:
df['msPlayed'] = df['msPlayed'].astype(float)

In [130]:
df['endTime'] = pd.to_datetime(df['endTime'])

In [131]:
#df['startTime'] = 0
for idx, row in tqdm(enumerate(df.values)):
    res = pd.to_datetime(df.iloc[idx].endTime + datetime.timedelta(milliseconds=-df.iloc[idx]['msPlayed']))
    df.loc[idx, "startTime"] = res


0it [00:00, ?it/s]
230it [00:00, 2294.91it/s]
478it [00:00, 2345.29it/s]
703it [00:00, 2315.34it/s]
949it [00:00, 2354.68it/s]
1194it [00:00, 2380.79it/s]
1443it [00:00, 2410.58it/s]
1698it [00:00, 2449.51it/s]
1943it [00:00, 2448.63it/s]
2199it [00:00, 2478.77it/s]
2450it [00:01, 2487.81it/s]
2693it [00:01, 2444.18it/s]
2934it [00:01, 2393.09it/s]
3171it [00:01, 2366.08it/s]
3410it [00:01, 2371.47it/s]
3665it [00:01, 2422.07it/s]
3916it [00:01, 2446.15it/s]
4166it [00:01, 2459.76it/s]
4412it [00:01, 2416.47it/s]
4654it [00:01, 2394.04it/s]
4894it [00:02, 2356.47it/s]
5130it [00:02, 2351.51it/s]
5369it [00:02, 2361.19it/s]
5606it [00:02, 2357.64it/s]
5855it [00:02, 2393.06it/s]
6095it [00:02, 2379.42it/s]
6334it [00:02, 2354.71it/s]
6580it [00:02, 2385.22it/s]
6836it [00:02, 2432.86it/s]
7080it [00:02, 2410.32it/s]
7322it [00:03, 2409.09it/s]
7565it [00:03, 2414.70it/s]
7818it [00:03, 2444.54it/s]
8063it [00:03, 2378.29it/s]
8302it [00:03, 2337.91it/s]
8537it [00:03, 2312.84it/s]
8962

In [132]:
df.head(2)

,endTime,track_id,track_name,track_artist,genres,msPlayed,audio_data,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,startTime
0,2021-12-30 19:06:00,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165.0,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,0.331,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4,2021-12-30 19:05:21.835
1,2021-12-30 19:06:00,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224.0,"[{'danceability': 0.798, 'energy': 0.574, 'key...",0.798,0.574,1,...,0.309,155.964,audio_features,4eL3XeuGaEoVT8ttDh3hwY,spotify:track:4eL3XeuGaEoVT8ttDh3hwY,https://api.spotify.com/v1/tracks/4eL3XeuGaEoV...,https://api.spotify.com/v1/audio-analysis/4eL3...,138587,4,2021-12-30 19:05:55.776


In [133]:
df.to_csv(file_out, index=False)

In [ ]:
df